In [13]:
""" ------------------- Library Imports ------------------- """
import pandas as pd
import requests
import json
import subprocess


In [7]:
def get_pub_ip_addr_and_port(name):
    temp = !kubectl get services --all-namespaces
    node_port = ""
    for line in temp:
        if name in line:
            parts = line.split()
            if len(parts) >= 6:
                node_port = parts[5].split(':')[1].split('/')[0]  # Extract NodePort
    temp = !kubectl get pods -o wide | findstr /R {name}
    node_name = temp[0].split()[6]  # Node name is the 7th column (index 6)
    temp = !kubectl get nodes -o wide | findstr /R {node_name}
    pub_ip = temp[0].split()[6]  # Public IP is the
    return f"{pub_ip}:{node_port}"

def get_inf_ipaddress():
    # Get all pods in all namespaces in JSON format
    result = subprocess.run(
        ["kubectl", "get", "pods", "-A", "-o", "json"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        check=True
    )
    pods_info = json.loads(result.stdout)
    pod_ips = []
    for item in pods_info.get("items", []):
        metadata = item.get("metadata", {})
        name = metadata.get("name", "")
        status = item.get("status", {})
        pod_ip = status.get("podIP")
        if "api-inferencia" in name and pod_ip:
            pod_ips.append(pod_ip)
    return pod_ips

def copy_file_to_docker(file_path, docker_path):
    """Copy a file from local to GCP VM and then into the Docker container."""
    filename = file_path.split("/")[-1] if "/" in file_path else file_path.split("\\")[-1]
    app_data_path = !dir "%APPDATA%"
    app_data_path = app_data_path[3].split()[-1]
    app_data_path = app_data_path.replace("\\", "/")
    username = app_data_path.split("/")[2]
    # Copy file to VM
    !gcloud compute scp --zone=us-central1-a {file_path} load-testing-instance:/home/{username}
    # Copy file from VM to Docker container
    copy_command = f"docker cp /home/{username}/{filename} load-testing-container:{docker_path}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{copy_command}"

def run_docker_command(inner_command):
    """
    Run a command inside load-testing-container on the GCP VM via SSH using docker exec.
    """
    docker_exec_template = "docker exec load-testing-container bash -c '{cmd}'"
    docker_command = docker_exec_template.format(cmd=inner_command)
    ssh_command = f"gcloud compute ssh load-testing-instance --zone=us-central1-a --command=\"{docker_command}\""
    !{ssh_command}

def copy_dir_to_docker(dir_path, docker_path):
    """Copy a directory from local to GCP VM and then into the Docker container."""
    dir_name = dir_path.split("/")[-1] if "/" in dir_path else dir_path.split("\\")[-1]
    username = dir_path.split("/")[2] if "/" in dir_path else dir_path.split("\\")[-2]
    # Clean up the VM directory 
    rm_command = f"sudo rm -rf /home/{username}/{dir_name}"  # Clean up the VM directory after copying
    docker_rm_command = f"docker exec load-testing-container rm -rf {docker_path}{dir_name}"  # Clean up the Docker directory
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{rm_command}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{docker_rm_command}"
    # Copy directory to VM
    print(f"gcloud compute scp --zone=us-central1-a --recurse {dir_path} load-testing-instance:/home/{username}/")
    !gcloud compute scp --zone=us-central1-a --recurse {dir_path} load-testing-instance:/home/{username}/
    # Copy directory from VM to Docker container
    copy_command = f"docker cp /home/{username}/{dir_name} load-testing-container:{docker_path}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{copy_command}"

def run_vm_command(command):
    """
    Run a command on the GCP VM via SSH.
    """
    ssh_command = f"gcloud compute ssh load-testing-instance --zone=us-central1-a --command=\"{command}\""
    !{ssh_command}

def copy_docker_file_to_local(docker_path, local_path):
    """Copy a file or directory from the Docker container to local."""
    # Get username from VM
    app_data_path = !dir "%APPDATA%"
    app_data_path = app_data_path[3].split()[-1]
    app_data_path = app_data_path.replace("\\", "/")
    username = app_data_path.split("/")[2]
    filename = docker_path.split("/")[-1] if "/" in docker_path else docker_path.split("\\")[-1]
    
    # Copy file from Docker container to VM home directory
    copy_command = f"docker cp load-testing-container:{docker_path} /home/{username}/{filename}"
    !gcloud compute ssh load-testing-instance --zone=us-central1-a --command="{copy_command}"
    # Copy file from VM to local
    copy_command = f"gcloud compute scp --zone=us-central1-a load-testing-instance:/home/{username}/{filename} {local_path}"
    !{copy_command}

In [8]:
# Get the public IP address and port for the update and inference services
ip_address_update = "http://" + get_pub_ip_addr_and_port("upd")
ip_address_inference = "http://" + get_pub_ip_addr_and_port("inf")
# ip_address_update = "http://localhost/update-service"
# ip_address_inference = "http://localhost/inference-service"
print(f"Update Service IP: {ip_address_update}")
print(f"Inference Service IP: {ip_address_inference}")

Update Service IP: http://34.10.166.82:32002
Inference Service IP: http://34.56.104.203:32001


In [42]:
# List of experiments to run
experiments = [
    # ["AGR_g-HT-old-1000", "AGR_g_real_test.csv"],
    # ["AGR_g-HT-old-2000", "AGR_g_real_test.csv"],
    # ["AGR_g-HT-old-10000", "AGR_g_real_test.csv"],
    # ["AGR_g-HT-1new-0", "AGR_g_real_test.csv"],
    # ["AGR_a-HT-old-2000", "AGR_a_real_test.csv"],
    # ["AGR_a-HT-old-10000", "AGR_a_real_test.csv"],
    # ["AGR_a-HT-old-1000", "AGR_a_real_test.csv"],
    # ["AGR_a-HT-old-1000", "AGR_a_real_test.csv"],
    # ["AGR_a-HT-old-1000", "AGR_a_real_test.csv"],
    ["AGR_a-HT-1new-0", "AGR_a_real_test.csv"],
]
loads = 1000
experiment = experiments[0]
experiment_name = experiment[0]  # Use the first experiment name
dataset_name = experiment[1]  # Use the first dataset name
mlflow_service_uri = "http://mlflow:5000"
kafka_service_uri = "kafka-service.kafka:9092"
inf_ipaddress = get_inf_ipaddress()
inf_load_urls = ["http://" + ip + ":5001/load" for ip in inf_ipaddress]
model_update_rate = experiment[0].split("-")[-1]  # Extracting the rate from the experiment name 

# FOR INFERENCE KAFKA
# url_kafka = f'{ip_address_inference}/kafka-uri'
# r_kafka = requests.post(url_kafka, json=(kafka_service_uri))
# print(f"Kafka URI INFERENCE configured with response: {r_kafka.status_code}")
# print('Curl for INFERENCE KAFKA:')
# print(f"curl -X POST {url_kafka} -H 'Content-Type: application/json' -d '{json.dumps(kafka_service_uri)}'")

# FOR INFERENCE MLFLOW
# url_kafka = f'{ip_address_inference}/mlflow-uri'
# r_kafka = requests.post(url_kafka, json=(mlflow_service_uri))
# print(f"MLFLOW URI INFERENCE configured with response: {r_kafka.status_code}")
# print('Curl for INFERENCE MLFLOW:')
# print(f"curl -X POST {ip_address_inference}/mlflow-uri -H 'Content-Type: application/json' -d '{json.dumps(mlflow_service_uri)}'")

# GIVE INFERENCE PODS
update_to_pods = f"{ip_address_update}/pods"
r_inf_pods = requests.post(update_to_pods, json=inf_load_urls)
print(f"Pods loaded with response: {r_inf_pods.status_code}")
# print('Curl for UPDATE PODS:')
# print(f"curl -X POST {update_to_pods} -H 'Content-Type: application/json' -d '{json.dumps(inf_load_urls)}'")


# # Reset Counter
url_count = f'{ip_address_update}/count'
r_count = requests.post(url_count, json=(0))
print(f"Counter reset with response: {r_count.status_code}")
# print('Curl for UPDATE COUNT:')
# print(f"curl -X POST {url_count} -H 'Content-Type: application/json' -d '0'")


# # Configure Rate
url_rate = f'{ip_address_update}/rate'
r_rate = requests.post(url_rate, json=(model_update_rate))
print(f"Rate configured with response: {r_rate.status_code}")
# print('Curl for UPDATE RATE:')
# print(f"curl -X POST {url_rate} -H 'Content-Type: application/json' -d '{json.dumps(model_update_rate)}'")


url_mlflow = f'{ip_address_update}/mlflow-uri'
r_mlflow = requests.post(url_mlflow, json=(mlflow_service_uri))
print(f"MLflow URI configured with response: {r_mlflow.status_code}")
# print('Curl for UPDATE MLFLOW:')
# print(f"curl -X POST {url_mlflow} -H 'Content-Type: application/json' -d '{json.dumps(mlflow_service_uri)}'")


url_kafka = f'{ip_address_update}/kafka-uri'
r_kafka = requests.post(url_kafka, json=(kafka_service_uri))
print(f"Kafka URI configured with response: {r_kafka.status_code}")
# print('Curl for UPDATE KAFKA:')
# print(f"curl -X POST {url_kafka} -H 'Content-Type: application/json' -d '{json.dumps(kafka_service_uri)}'")


url_load = f'{ip_address_update}/load'
exp_name = experiment[0]  # Use the first experiment name
r_load = requests.post(url_load, json=(exp_name))
print(f"Experiment {exp_name} loaded with response: {r_load.status_code}")
# print('Curl for UPDATE LOAD:')
# print(f"curl -X POST {url_load} -H 'Content-Type: application/json' -d '{json.dumps(exp_name)}'")


Pods loaded with response: 200
Counter reset with response: 200
Rate configured with response: 200
MLflow URI configured with response: 200
Kafka URI configured with response: 200
Experiment AGR_a-HT-1new-0 loaded with response: 200


In [10]:
# [SKIP] Simple testing of the inference service
df = pd.read_csv(r'C:\Users\muhammad.aulia\large-scale-online-learning\Datasets\real_usage\AGR_g_real_test.csv')
count = 0
for idx, row in df.iterrows():
    # Convert row to list and prepare data
    data = row.tolist()
    
    # Send data to your prediction endpoint
    response = requests.post(ip_address_inference + "/predict", json=data)
    
    if response.status_code == 200:
        prediction_result = response.json()
        print(f"Row {idx}: Prediction = {prediction_result}")
    else:
        print(f"Row {idx}: Error - {response.status_code}")
    count += 1
    if count >= 5000:
        break

Row 0: Prediction = 0.02326669978609882
Row 1: Prediction = 0.9108201652401648
Row 2: Prediction = 0.02326669978609882
Row 3: Prediction = 0.0
Row 4: Prediction = 0.0
Row 5: Prediction = 0.7165180280047907
Row 6: Prediction = 0.02326669978609882
Row 7: Prediction = 0.02326669978609882
Row 8: Prediction = 0.02326669978609882
Row 9: Prediction = 0.815006027942381
Row 10: Prediction = 0.0
Row 11: Prediction = 0.02326669978609882
Row 12: Prediction = 0.715050910355845
Row 13: Prediction = 0.02326669978609882
Row 14: Prediction = 0.02326669978609882
Row 15: Prediction = 0.02326669978609882
Row 16: Prediction = 0.0
Row 17: Prediction = 0.0
Row 18: Prediction = 0.9197849396351291
Row 19: Prediction = 0.0
Row 20: Prediction = 0.02326669978609882
Row 21: Prediction = 0.5180240554112743
Row 22: Prediction = 0.02326669978609882
Row 23: Prediction = 0.0
Row 24: Prediction = 0.9052542451937076
Row 25: Prediction = 0.7444504820951255
Row 26: Prediction = 0.02326669978609882
Row 27: Prediction = 0.02

KeyboardInterrupt: 

In [43]:
# FOR LOCUST
# locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py
# locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py --users 1 --spawn-rate 1 --headless --csv=result_testing
# [BACKGROUND RUN]
# nohup locust -f MLOps-Architecture/Serialization_Datasets/locust-testing.py --users 1 --spawn-rate 1 --headless --csv=result_testing > locust.log 2>&1 &

import re

substitutions = {
    # C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\MLOps-Architecture\Serialization_Datasets\testing.ipynb
    # C:\Users\adilm\OneDrive\Documents\large-scale-online-learning\Datasets\real_usage
    # go to real_usage using relative path
    "dataset_dir": "Datasets/real_usage/",
    "file_name": dataset_name,
    # "host1": "http://localhost/inference-service"  
    "host1": ip_address_inference
}

for var, value in substitutions.items():
    # For string values, ensure quotes in the replacement
    replacement = f'{var} = "{value}"'
    # Use sed to replace the line in locust-test.py
    file_path = 'locust-testing.py'

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    for var, value in substitutions.items():
        # Replace lines like: var = "old_value"
        content = re.sub(
            rf'^{var}\s*=.*$', 
            f'{var} = "{value}"', 
            content, 
            flags=re.MULTILINE
        )
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(content)

In [ ]:
# [SKIP] EXPORT VARIABLES
variables = {
    "experiments": str(experiments),
    "experiment": str(experiment),
    "mlflow_service_uri": mlflow_service_uri,
    "kafka_service_uri": kafka_service_uri,
    "inf_ipaddress": str(inf_ipaddress),
    "inf_load_urls": str(inf_load_urls),
    "model_update_rate": model_update_rate
}

with open("etc/variables.txt", "w") as f:
    for key, value in variables.items():
        f.write(f"{key}={value}\n")

In [44]:
# COPY FILE LOCUST OR CHECK KAFKA LAG 
copy_file_to_docker("locust-testing.py", "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/")
# copy_file_to_docker("check-kafka-lag.py", "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/")



locust-testing.py         | 3 kB |   3.4 kB/s | ETA: 00:00:00 | 100%


In [20]:
# COPY GOOGLE CLOUD

app_data_path = !dir "%APPDATA%"
!rmdir /s /q %APPDATA%\gcloud\logs
app_data_path = app_data_path[3].split()[-1]
app_data_path = app_data_path.replace("\\", "/")
copy_dir_to_docker(f'{app_data_path}/gcloud', "/root/.config/gcloud")

gcloud compute scp --zone=us-central1-a --recurse C:/Users/muhammad.aulia/AppData/Roaming/gcloud load-testing-instance:/home/muhammad.aulia/

.last_opt_in_prompt.yaml  | 0 kB |   0.0 kB/s | ETA: 00:00:00 | 100%

.last_survey_prompt.yaml  | 0 kB |   0.0 kB/s | ETA: 00:00:00 | 100%

.last_update_check.json   | 0 kB |   0.2 kB/s | ETA: 00:00:00 | 100%

access_tokens.db          | 4 kB |   4.0 kB/s | ETA: 00:00:02 |  33%
access_tokens.db          | 12 kB |  12.0 kB/s | ETA: 00:00:00 | 100%

active_config             | 0 kB |   0.0 kB/s | ETA: 00:00:00 | 100%

application_default_crede | 0 kB |   0.4 kB/s | ETA: 00:00:00 | 100%

resource.cache            | 4 kB |   4.0 kB/s | ETA: 00:00:02 |  33%
resource.cache            | 12 kB |  12.0 kB/s | ETA: 00:00:00 | 100%

config_default            | 0 kB |   0.1 kB/s | ETA: 00:00:00 | 100%

credentials.db            | 4 kB |   4.0 kB/s | ETA: 00:00:02 |  33%
credentials.db            | 12 kB |  12.0 kB/s | ETA: 00:00:00 | 100%

default_configs.db

pscp: C:/Users/muhammad.aulia/AppData/Roaming/gcloud/logs/2025.08.23/12.44.05.829656.log: Cannot open file

ERROR: (gcloud.compute.scp) [C:\Users\muhammad.aulia\AppData\Local\Google\Cloud SDK\google-cloud-sdk\bin\sdk\pscp.exe] exited with return code [1].


In [ ]:
# Check copy file
path = "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets"
run_docker_command("cat " + path + "/locust-testing.py")
# run_docker_command("cat " + path + "/check-kafka-lag.py")

In [21]:
# Check GCloud
projects = !gcloud config get-value project
set_project_command = f"gcloud config set project {projects[0]}"
get_kubectl_command = "gcloud container clusters get-credentials two-node-cluster --zone us-central1-a"
run_docker_command(set_project_command)
run_docker_command(get_kubectl_command)
run_docker_command("kubectl get pods")
# run_docker_command("gcloud config get-value project")

Updated property [core/project].
Fetching cluster endpoint and auth data.
kubeconfig entry generated for two-node-cluster.


NAME                              READY   STATUS    RESTARTS   AGE
api-inferencia-685b667b65-2tj5d   1/1     Running   0          71m
api-inferencia-685b667b65-7l4nq   1/1     Running   0          71m
api-inferencia-685b667b65-g9xck   1/1     Running   0          71m
api-inferencia-685b667b65-xch42   1/1     Running   0          71m
api-update-d44fddd4c-5ch7f        1/1     Running   0          71m
mlflow-6cf7458897-r5t6w           1/1     Running   0          71m


In [ ]:
# Delete all copied files in kubernetes and RUN THE TEST
pods = !kubectl get pods | findstr /R "upd"
!kubectl exec {pods[0].split()[0]} -- rm /app/message_log.csv
!kubectl exec {pods[0].split()[0]} -- rm /app/model_upload_latency.csv
!kubectl exec {pods[0].split()[0]} -- rm /app/for_auc.csv
pods = !kubectl get pods | findstr /R "inf"
for pod in pods:
    # Delete all files matching load_model*.csv at the end of the file in /app, suppress error if not found
    !kubectl exec {pods[0].split()[0]} -- bash -c "rm -f load_model*.gz"
    !kubectl exec {pod.split()[0]} -- bash -c "rm -f load_model*.csv"
    !kubectl exec {pod.split()[0]} -- bash -c "rm -f api_inference_*.log"
run_docker_command("rm -rf /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/kafka_lag_log.csv")
run_docker_command("rm -rf /app/large-scale-online-learning/locust.log")

# RUN THE TEST
locust_path = "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/locust-testing.py"
check_kafka_lag_dir_path = "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets"
locust_command = f"locust -f {locust_path} --users {loads} --spawn-rate {loads} --headless --csv=result_testing"
run_docker_command(f"cd {check_kafka_lag_dir_path} && nohup python3 check-kafka-lag.py > kafka_lag.log 2>&1 &")
run_docker_command(f"cd /app/large-scale-online-learning/ && source ../.python-venv/bin/activate && nohup {locust_command} > locust.log 2>&1 &")

rm: cannot remove '/app/message_log.csv': No such file or directory
command terminated with exit code 1
rm: cannot remove '/app/model_upload_latency.csv': No such file or directory
command terminated with exit code 1
rm: cannot remove '/app/for_auc.csv': No such file or directory
command terminated with exit code 1


In [48]:
# CHECK IF LOAD TESTING IS WORKING
# run_docker_command("cat /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/kafka_lag_log.csv | tail -n 10")
run_docker_command("cat /app/large-scale-online-learning/locust.log | tail -n 10")

# clean kafka lag log
# run_docker_command("rm -rf /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/kafka_lag_log.csv")
# run_docker_command("rm -rf /app/large-scale-online-learning/locust.log")

POST     /predict                                                                      494029 494029(100.00%) |    751      12    1881    710 | 1323.80     1323.80
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated                                                                    494029 494029(100.00%) |    751      12    1881    710 | 1323.80     1323.80

Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
POST     /predict                                                                      494029 494029(100.00%) |    751      12    1881    710 | 1323.80     1323.80
--------|---------------

In [39]:
# STOP THE PROCESS KAFKA LAG AND LOCUST TESTING
run_docker_command("pkill -f check-kafka-lag.py")
run_docker_command("pkill -f locust-testing.py")

In [42]:
# GET INFERENCE RESULTS | GET UPDATE RESULTS | GET MLFLOW RESULTS | GET LOAD TESTING RESULTS
inference_pods = !kubectl get pods | findstr /R "inf"
if inference_pods:
    filenames = !kubectl exec {inference_pods[0].split()[0]} -- ls | findstr /R "load_model"


for filename in filenames:
    !kubectl exec {inference_pods[0].split()[0]} -- tar -czf /app/{filename}.tar.gz /app/{filename}
    experiment_folder = f"experiment-results\\{experiment_name + '-' + str(loads) + 'VU'}\\inference-results"
    !mkdir {experiment_folder}
    !kubectl cp default/{inference_pods[0].split()[0]}:/app/{filename}.tar.gz {experiment_folder}/{filename}.tar.gz
# GET UPDATE RESULTS
pods = !kubectl get pods | findstr /R "upd"
!kubectl exec {pods[0].split()[0]} -- tar -czf /app/for_auc.tar.gz /app/for_auc.csv
!kubectl exec {pods[0].split()[0]} -- tar -czf /app/message_log.tar.gz /app/message_log.csv
!kubectl exec {pods[0].split()[0]} -- tar -czf /app/model_upload_latency.tar.gz /app/model_upload_latency.csv
experiment_folder = f"experiment-results\\{experiment_name + '-' + str(loads) + 'VU'}\\update-results"
!mkdir {experiment_folder}
run_docker_command(f"kubectl cp default/{pods[0].split()[0]}:/app/for_auc.tar.gz app/for_auc.tar.gz")
copy_docker_file_to_local("/app/for_auc.tar.gz", f"{experiment_folder}/for_auc.tar.gz")
run_docker_command(f"kubectl cp default/{pods[0].split()[0]}:/app/message_log.tar.gz app/message_log.tar.gz")
copy_docker_file_to_local("/app/message_log.tar.gz", f"{experiment_folder}/message_log.tar.gz")
run_docker_command(f"kubectl cp default/{pods[0].split()[0]}:/app/model_upload_latency.tar.gz app/model_upload_latency.tar.gz")
copy_docker_file_to_local("/app/model_upload_latency.tar.gz", f"{experiment_folder}/model_upload_latency.tar.gz")
# GET MLFLOW RESULT
pods = !kubectl get pods | findstr /R "mlflow"
!kubectl exec {pods[0].split()[0]} -- tar -czf /mlartifacts/mlflow-results.tar.gz /mlartifacts/1
!kubectl exec {pods[0].split()[0]} -- tar -czf /mlartifacts/mlflow-db.tar.gz /mlartifacts/mlflow.db
experiment_folder = f"experiment-results\\{experiment_name + '-' + str(loads) + 'VU'}\\mlflow-results"
!kubectl cp default/{pods[0].split()[0]}:/mlartifacts/mlflow-results.tar.gz {experiment_folder}/mlflow-results.tar.gz
!kubectl cp default/{pods[0].split()[0]}:/mlartifacts/mlflow-db.tar.gz {experiment_folder}/mlflow-db.tar.gz 
# GET LOAD TESTING RESULT
docker_path = "/app/large-scale-online-learning/"
docker_path_kafka_log = "/app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/"
experiment_folder = f"experiment-results\\{experiment_name + '-' + str(loads) + 'VU'}\\load-test-results"

import os
os.makedirs(experiment_folder, exist_ok=True)

copy_docker_file_to_local(f"{docker_path}result_testing_stats_history.csv", experiment_folder)
copy_docker_file_to_local(f"{docker_path}result_testing_exceptions.csv", experiment_folder)
copy_docker_file_to_local(f"{docker_path}result_testing_failures.csv", experiment_folder)
copy_docker_file_to_local(f"{docker_path}result_testing_stats.csv", experiment_folder)
copy_docker_file_to_local(f"{docker_path_kafka_log}kafka_lag_log.csv", experiment_folder)


tar: Removing leading `/' from member names


tar: Removing leading `/' from member names


tar: Removing leading `/' from member names
A subdirectory or file experiment-results\AGR_g-HT-old-10000-30000VU\inference-results already exists.
tar: Removing leading `/' from member names
A subdirectory or file experiment-results\AGR_g-HT-old-10000-30000VU\inference-results already exists.


tar: Removing leading `/' from member names


tar: Removing leading `/' from member names
A subdirectory or file experiment-results\AGR_g-HT-old-10000-30000VU\inference-results already exists.
tar: Removing leading `/' from member names
tar: Removing leading `/' from member names
tar: Removing leading `/' from member names


tar: Removing leading `/' from member names

for_auc.tar.gz            | 32 kB |  32.0 kB/s | ETA: 00:01:56 |   0%
for_auc.tar.gz            | 128 kB |  64.0 kB/s | ETA: 00:00:56 |   3%
for_auc.tar.gz            | 576 kB | 192.0 kB/s | ETA: 00:00:16 |  15%
for_auc.tar.gz            | 1056 kB | 264.0 kB/s | ETA: 00:00:10 |  28%
for_auc.tar.gz            | 1824 kB | 364.8 kB/s | ETA: 00:00:05 |  48%
for_auc.tar.gz            | 2336 kB | 389.3 kB/s | ETA: 00:00:03 |  62%
for_auc.tar.gz            | 2880 kB | 411.4 kB/s | ETA: 00:00:02 |  76%
for_auc.tar.gz            | 3712 kB | 464.0 kB/s | ETA: 00:00:00 |  98%
for_auc.tar.gz            | 3754 kB | 469.4 kB/s | ETA: 00:00:00 | 100%
tar: Removing leading `/' from member names

message_log.tar.gz        | 32 kB |  32.0 kB/s | ETA: 00:10:28 |   0%
message_log.tar.gz        | 64 kB |  32.0 kB/s | ETA: 00:10:27 |   0%
message_log.tar.gz        | 128 kB |  42.7 kB/s | ETA: 00:07:49 |   0%
message_log.tar.gz        | 160 kB |  40.0 kB/s | ETA: 

tar: Removing leading `/' from member names
tar: Removing leading `/' from member names


tar: Removing leading `/' from member names
tar: Removing leading `/' from member names

result_testing_stats_hist | 32 kB |  32.0 kB/s | ETA: 00:00:07 |  11%
result_testing_stats_hist | 64 kB |  64.0 kB/s | ETA: 00:00:03 |  23%
result_testing_stats_hist | 269 kB | 269.8 kB/s | ETA: 00:00:00 | 100%

result_testing_exceptions | 0 kB |   0.0 kB/s | ETA: 00:00:00 | 100%

result_testing_failures.c | 0 kB |   0.3 kB/s | ETA: 00:00:00 | 100%

result_testing_stats.csv  | 0 kB |   0.6 kB/s | ETA: 00:00:00 | 100%

kafka_lag_log.csv         | 0 kB |   0.5 kB/s | ETA: 00:00:00 | 100%


In [31]:
# check file size in /app
pods = !kubectl get pods | findstr /R "upd"
!kubectl exec {pods[0].split()[0]} -- ls -lh /app

total 114M
-rw-rw-rw- 1 root root 577K Aug 19 04:35 AGR_a_first_train.csv
-rw-rw-rw- 1 root root 577K Aug 19 04:35 AGR_g_first_train.csv
-rw-rw-rw- 1 root root 9.1K Aug 19 04:34 API_update_V8.1.py
-rw-r--r-- 1 root root 456K Aug 19 06:02 api_update.log
-rw-r--r-- 1 root root  16M Aug 19 06:03 for_auc.csv
-rw-r--r-- 1 root root 3.6M Aug 19 06:16 for_auc.tar.gz
drwx------ 2 root root  16K Aug 19 02:02 lost+found
-rw-r--r-- 1 root root  74M Aug 19 06:03 message_log.csv
-rw-r--r-- 1 root root  20M Aug 19 06:16 message_log.tar.gz
-rw-r--r-- 1 root root  12K Aug 19 06:03 model_upload_latency.csv
-rw-r--r-- 1 root root 3.2K Aug 19 06:16 model_upload_latency.tar.gz


In [43]:
# Delete all copied files in kubernetes
pods = !kubectl get pods | findstr /R "upd"
!kubectl exec {pods[0].split()[0]} -- rm /app/message_log.csv
!kubectl exec {pods[0].split()[0]} -- rm /app/model_upload_latency.csv
!kubectl exec {pods[0].split()[0]} -- rm /app/for_auc.csv
pods = !kubectl get pods | findstr /R "inf"
for pod in pods:
    # Delete all files matching load_model*.csv at the end of the file in /app, suppress error if not found
    !kubectl exec {pods[0].split()[0]} -- bash -c "rm -f load_model*.gz"
    !kubectl exec {pod.split()[0]} -- bash -c "rm -f load_model*.csv"
    !kubectl exec {pod.split()[0]} -- bash -c "rm -f api_inference_*.log"
run_docker_command("rm -rf /app/large-scale-online-learning/MLOps-Architecture/Serialization_Datasets/kafka_lag_log.csv")
run_docker_command("rm -rf /app/large-scale-online-learning/locust.log")